In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
%matplotlib inline

train = pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [2]:
sexArray = ['male','female']
def getGender(s):
    return sexArray.index(s)

In [3]:
train['Sex'] = train['Sex'].apply(lambda c:getGender(c))

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(6), object(4)
memory usage: 90.5+ KB


In [5]:
train['Title'] = train['Name'].str.extract(' ([A-za-z]+)\.')

In [6]:
train.Title.value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Col           2
Major         2
Mlle          2
Countess      1
Ms            1
Lady          1
Jonkheer      1
Don           1
Mme           1
Capt          1
Sir           1
Name: Title, dtype: int64

In [7]:
titleArray = ['Mr','Miss','Mrs','Master','Dr','Rev','Col','Major','Mlle','Countess','Ms','Lady','Jonkheer',
              'Don','Mme','Capt','Sir']
def getTitleIndex(s):
    if s in titleArray:
        return titleArray.index(s)
    else:
        return -1

In [8]:
train['Title'] = train['Title'].apply(lambda c:getTitleIndex(c))

In [9]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,S,2
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,NaN,S,0


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 13 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
Title          891 non-null int64
dtypes: float64(2), int64(7), object(4)
memory usage: 97.5+ KB


In [11]:
def getAgeGroup(age):
    if age >0 and age <=10:
        return 0
    elif age >10 and age <= 20:
        return 1
    elif age >20 and age <= 30:
        return 3
    elif age >31 and age <=45:
        return 4
    else:
        return 5

In [12]:
train['Age'].median()

28.0

In [13]:
train['Age'].mean()

29.69911764705882

In [14]:
train['Age'].fillna(train['Age'].median(),inplace = True)

In [15]:
train['AgeGroup'] = train['Age'].apply(lambda x:getAgeGroup(x))

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 14 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
Title          891 non-null int64
AgeGroup       891 non-null int64
dtypes: float64(2), int64(8), object(4)
memory usage: 104.4+ KB


In [17]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,AgeGroup
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,NaN,S,0,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,C,2,4
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,S,2,4
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,NaN,S,0,4


In [18]:
embarkedArray = ['S','C','Q']
def getEmbarkedIndex(s):
    if s in embarkedArray:
        return embarkedArray.index(s)
    else:
        return -1

train['Embarked'].fillna('U',inplace = True)
train['Embarked'] = train['Embarked'].apply(lambda c:getEmbarkedIndex(c))

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 14 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null int64
Title          891 non-null int64
AgeGroup       891 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 104.4+ KB


In [20]:
drop_features = ['PassengerId','Survived','Name','Ticket','Fare','Cabin','Age']

In [21]:
x = train.drop(drop_features,axis=1)

In [22]:
x.head()

,Pclass,Sex,SibSp,Parch,Embarked,Title,AgeGroup
0,3,0,1,0,0,0,3
1,1,1,1,0,1,2,4
2,3,1,0,0,0,1,3
3,1,1,1,0,0,2,4
4,3,0,0,0,0,0,4


In [23]:
y = train.Survived

In [24]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [25]:
test = pd.read_csv('test.csv')

In [26]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [27]:
test['Sex'] = test['Sex'].apply(lambda c:getGender(c))
test['Title'] = test['Name'].str.extract(' ([A-za-z]+)\.')
test['Title'] = test['Title'].apply(lambda c:getTitleIndex(c))

In [28]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S,2
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S,2


In [29]:
test['AgeGroup'] = test['Age'].apply( lambda c:getAgeGroup(c))

In [30]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,AgeGroup
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q,0,4
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S,2,5
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q,0,5
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S,0,3
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S,2,3


In [31]:
test['Age'].median()

27.0

In [32]:
test['Age'].mean()

30.272590361445783

In [33]:
test['Age'].fillna(test['Age'].median(),inplace = True)
test['Embarked'] = test['Embarked'].apply(lambda c:getEmbarkedIndex(c))

In [34]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 13 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null int64
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null int64
Title          418 non-null int64
AgeGroup       418 non-null int64
dtypes: float64(2), int64(8), object(3)
memory usage: 45.7+ KB


In [35]:
drop_featuresY = ['PassengerId','Name','Ticket','Fare','Cabin','Age']
test = test.drop(drop_featuresY,axis=1)

In [36]:
test.head()

,Pclass,Sex,SibSp,Parch,Embarked,Title,AgeGroup
0,3,0,0,0,2,0,4
1,3,1,1,0,0,2,5
2,2,0,0,0,2,0,5
3,3,0,0,0,0,0,3
4,3,1,1,1,0,2,3


In [37]:
from sklearn import tree

In [38]:
model = tree.DecisionTreeClassifier()

In [39]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Sex         891 non-null int64
SibSp       891 non-null int64
Parch       891 non-null int64
Embarked    891 non-null int64
Title       891 non-null int64
AgeGroup    891 non-null int64
dtypes: int64(7)
memory usage: 55.7 KB


In [40]:
model.fit(x,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [41]:
pred = model.predict(test)

In [42]:
my_submission = pd.DataFrame({'PassengerId': test.index+892, 'Survived': pred})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [50]:
#import graphviz
#dot_tree = tree.export_graphviz(model,out_file=None)
#graph = graphviz.Source(dot_tree)
#graph

In [63]:
from sklearn import linear_model

In [68]:
lr = linear_model.LogisticRegression()

In [69]:
lr.fit(x,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [66]:
pred2 = lr.predict(test)

In [70]:
my_submission = pd.DataFrame({'PassengerId': test.index+892, 'Survived': pred2})
# you could use any filename. We choose submission here
my_submission.to_csv('submission2.csv', index=False)

In [71]:
from sklearn.svm import SVC

In [73]:
sv = SVC()
sv.fit(x,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [74]:
pred3 = sv.predict(test)

In [76]:
my_submission = pd.DataFrame({'PassengerId': test.index+892, 'Survived': pred2})
# you could use any filename. We choose submission here
my_submission.to_csv('submission3.csv', index=False)